In [1]:
import pandas as pd
import torch
import numpy as np
import fitz
from sentence_transformers import SentenceTransformer
import re
from spacy.lang.en import English

nlp = English()

nlp.add_pipe("sentencizer")
import random
from tqdm.auto import tqdm
import os
import requests

c:\ProgramData\anaconda3\envs\killme\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


Similarity Search


In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

embsdf = pd.read_csv("chunks_and_embs_df.csv")
# embsdf.head()

embsdf["embedding"] = embsdf["embedding"].apply(lambda x: np.fromstring(x.strip("[]"), sep=" "))
pac = embsdf.to_dict(orient="records")

embeddings = torch.tensor(np.stack(embsdf["embedding"].tolist(), axis=0), dtype=torch.float32).to(device=device)

embsdf

,page_number,sentence_chunk,chunk_word_count,chunk_token_count,embedding
0,0,"""Bhagavad-gita As It Is"" by His Divine Grace A...",213,353.75,"[0.0211626217, 0.0586190559, -0.0255278405, -0..."
1,3,Preface Originally I wrote Bhagavad-gétä As It...,266,449.50,"[0.0681742355, 0.0634085238, -0.00446655322, 0..."
2,3,Some of them said that it is greatly fortunate...,24,36.75,"[-0.00864421204, 0.097841993, -0.00656992709, ..."
3,4,original father of this movement is Lord Krish...,254,423.75,"[0.0603890792, 0.0354894325, -0.0110460483, 0...."
4,4,Such unauthorized commentary upon Bhagavad-gét...,132,197.50,"[0.0430899151, 0.0805046633, -0.0035992125, 0...."
...,...,...,...,...,...
1482,1048,"50, Orizaba, Ver./ Tel.+52 (2717) 14825 PERU L...",18,28.25,"[0.00698623061, -0.00365059334, -0.0355605371,..."
1483,1049,◆ Lima — Schell 634 Miraﬂores/ Tel.+51 (014) 4...,143,217.75,"[0.0495859608, 0.0401996486, -0.0269423407, 0...."
1484,1049,+593 (04) 308412 or 309420/ Fax: +564 302108/...,152,258.25,"[0.0377525911, -0.0121238222, -0.0390545651, 0..."
1485,1049,+1 (809) 647-3165/ Fax: +1 (868) 647-6809/ E-...,87,153.25,"[0.0195350163, 0.0349830948, -0.0256691165, 0...."


In [3]:
embsdf["embedding"]

0       [0.0211626217, 0.0586190559, -0.0255278405, -0...
1       [0.0681742355, 0.0634085238, -0.00446655322, 0...
2       [-0.00864421204, 0.097841993, -0.00656992709, ...
3       [0.0603890792, 0.0354894325, -0.0110460483, 0....
4       [0.0430899151, 0.0805046633, -0.0035992125, 0....
                              ...                        
1482    [0.00698623061, -0.00365059334, -0.0355605371,...
1483    [0.0495859608, 0.0401996486, -0.0269423407, 0....
1484    [0.0377525911, -0.0121238222, -0.0390545651, 0...
1485    [0.0195350163, 0.0349830948, -0.0256691165, 0....
1486    [0.00486746943, 0.0259503797, -0.0357118957, 0...
Name: embedding, Length: 1487, dtype: object

In [4]:
embeddings.shape

torch.Size([1487, 768])

MODEL

In [5]:
from sentence_transformers import util, SentenceTransformer

embedding_model = SentenceTransformer(model_name_or_path='all-mpnet-base-v2', device=device)

c:\ProgramData\anaconda3\envs\killme\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
query = "who was arjuna talking to"
print(f"Query : {query}")

#embed


query_emb = embedding_model.encode(query, convert_to_tensor=True).to("cuda")

dot_scores = util.dot_score(a=query_emb, b=embeddings)[0]

top_reults = torch.topk(dot_scores, k=1)
top_reults

Query : who was arjuna talking to


torch.return_types.topk(
values=tensor([0.7093], device='cuda:0'),
indices=tensor([152], device='cuda:0'))

In [7]:
# def func(query:str, model, embeddings: torch.tensor) -> str:




In [8]:
pacc = pac[top_reults.indices[0]]
pacc["sentence_chunk"]

'He wants to stop friendly talks.Talks between the master and the disciple are serious, and now Arjuna wants to talk very seriously before the recognized spiritual master.Krishna is therefore the original spiritual master of the science of Bhagavad-gétä, and Arjuna is the first disciple for understanding the Gétä.How Arjuna understands the Bhagavad-gétä is stated in the Gétä itself.And yet foolish mundane scholars explain that one need not submit to Krishna as a person, but to “the unborn within Krishna.”There is no difference between Krishna’s within and without.And one who has no sense of this understanding is the greatest fool in trying to understand Bhagavad-gétä.TEXT 8 Na ih Pa[PaXYaaiMa MaMaaPaNauÛa‚ ÛC^aek-MauC^aez<aiMaiNd]Yaa<aaMa( ) AvaPYa >aUMaavSaPaÒMa*Ö&'

In [9]:
# import textwrap

# def wrapped(text, wrap_length=10):
#     wrapped_Text = textwrap.fill(text, wrap_length)
#     print(wrapped_Text)


In [10]:
print(query)
print()
for score, idx in zip(top_reults[0], top_reults[1]):
    print(score)
    print(pac[idx]["sentence_chunk"])
    # print(f"page no: {pac[idx]["page_number"]}")
    print("\n")

who was arjuna talking to

tensor(0.7093, device='cuda:0')
He wants to stop friendly talks.Talks between the master and the disciple are serious, and now Arjuna wants to talk very seriously before the recognized spiritual master.Krishna is therefore the original spiritual master of the science of Bhagavad-gétä, and Arjuna is the first disciple for understanding the Gétä.How Arjuna understands the Bhagavad-gétä is stated in the Gétä itself.And yet foolish mundane scholars explain that one need not submit to Krishna as a person, but to “the unborn within Krishna.”There is no difference between Krishna’s within and without.And one who has no sense of this understanding is the greatest fool in trying to understand Bhagavad-gétä.TEXT 8 Na ih Pa[PaXYaaiMa MaMaaPaNauÛa‚ ÛC^aek-MauC^aez<aiMaiNd]Yaa<aaMa( ) AvaPYa >aUMaavSaPaÒMa*Ö&




In [11]:
embeddings[0]

tensor([ 2.1163e-02,  5.8619e-02, -2.5528e-02, -5.9816e-03, -2.7692e-02,
         1.8454e-02, -6.8008e-02,  1.3618e-02,  1.3084e-02,  6.9089e-03,
         3.0247e-02, -1.6177e-02, -1.0645e-02, -2.0372e-02,  2.1421e-02,
        -9.4767e-02,  1.7214e-02,  2.5570e-02, -1.1311e-02,  3.9779e-02,
        -8.8459e-03, -2.7681e-03, -1.4302e-02,  1.7964e-02,  6.1292e-03,
        -3.6292e-02,  3.0513e-02, -1.6677e-02, -2.0449e-02, -6.2248e-02,
        -3.1691e-02,  5.3905e-03,  5.9277e-03,  5.8910e-02,  2.7739e-06,
        -4.3372e-02, -4.2412e-03, -2.1236e-02, -3.4211e-02,  9.5382e-04,
        -2.1264e-02,  4.9878e-02,  4.0554e-02, -2.2962e-02, -2.0445e-02,
         9.0386e-02, -8.8574e-03,  6.5329e-02, -9.5708e-03,  4.7935e-02,
        -3.7647e-03, -1.1760e-01,  4.1330e-03, -3.5087e-02,  2.4020e-02,
         4.2913e-02, -3.1640e-02,  5.3709e-02,  1.4993e-02,  4.2173e-02,
        -4.8077e-02, -1.4930e-02, -1.1748e-02,  2.1585e-02,  6.1268e-02,
         3.8645e-02,  3.3878e-02,  2.0331e-02,  5.5

In [12]:
!nvidia-smi

Fri Sep 13 15:56:43 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.81                 Driver Version: 560.81         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   42C    P8              3W /   43W |     625MiB /   8188MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

LLMS :)

In [13]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.utils import is_flash_attn_2_available

from transformers import BitsAndBytesConfig
quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                         bnb_4bit_compute_dtype=torch.float16
                                        )


if (is_flash_attn_2_available()) and (torch.cuda.get_device_capability(0)[0] >= 8):
    attn_implementation = "flash-attention-2"

else:
    attn_implementation = "sdpa"


model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_id)


llmmodel = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_id,
                                                torch_dtype=torch.float16,
                                                quantization_config=quantization_config,
                                                # low_cpu_mem_usage=False,
                                                attn_implementation=attn_implementation)






`low_cpu_mem_usage` was None, now set to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 4/4 [00:23<00:00,  5.96s/it]


In [14]:

model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [15]:
llmmodel

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps

In [16]:
def abc(model:torch.nn.Module):
    return sum([param.numel() for param in llmmodel.parameters()])

In [17]:
abc(llmmodel)

4540600320

In [18]:
!nvidia-smi

Fri Sep 13 15:57:24 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.81                 Driver Version: 560.81         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   44C    P8              2W /   55W |    6401MiB /   8188MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [20]:

inp = "what was the name of divine conch of krishna"

In [21]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# checkpoint = "HuggingFaceH4/zephyr-7b-beta"
tokenizer = AutoTokenizer.from_pretrained(model_id)
# model = AutoModelForCausalLM.from_pretrained(checkpoint)  # You may want to use bfloat16 and/or move to GPU here

messages = [
    {
        "role": "user",
        "content": inp,
    }
    # {"role": "user", "content": inp},
 ]
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True) #, return_tensors="pt")
print(prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

what was the name of divine conch of krishna<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [22]:
# outputs = llmmodel.generate(tokenized_chat, max_new_tokens=100) 
# print(tokenizer.decode(outputs[0]))

inputid = tokenizer(prompt, return_tensors="pt").to("cuda")

outputs = llmmodel.generate(**inputid,
                            max_new_tokens=256)
outputs

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
c:\ProgramData\anaconda3\envs\killme\Lib\site-packages\transformers\models\llama\modeling_llama.py:660: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


tensor([[128000, 128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,
           2696,     25,   6790,    220,   2366,     18,    198,  15724,   2696,
             25,    220,   1627,  10263,    220,   2366,     19,    271, 128009,
         128006,    882, 128007,    271,  12840,    574,    279,    836,    315,
          30467,    390,    331,    315,  23975,    819,   3458, 128009, 128006,
          78191, 128007,    271,     40,   2846,    539,   8010,    315,    904,
           2038,    922,    264,    330,    614,    483,    390,    331,      1,
           5938,    449,  80409,     13,   4452,     11,    279,    390,    331,
          12811,    374,   3629,   5938,    449,  80409,    439,    264,   7891,
            315,    813,  30467,   7899,    477,    813,   3717,    311,    279,
          18435,     13, 128009]], device='cuda:0')

In [23]:
def retrieve_relevant_resources(query: str,
                                embeddings: torch.tensor,
                                model: SentenceTransformer=embedding_model,
                                n_resources_to_return: int=3,
                                print_time: bool=True):
    """
    Embeds a query with model and returns top k scores and indices from embeddings.
    """

    # Embed the query
    query_embedding = model.encode(query, 
                                   convert_to_tensor=True) 

    # Get dot product scores on embeddings
    # start_time = timer()
    dot_scores = util.dot_score(query_embedding, embeddings)[0]
    # end_time = timer()

    # if print_time:
        # print(f"[INFO] Time taken to get scores on {len(embeddings)} embeddings: {end_time-start_time:.5f} seconds.")

    scores, indices = torch.topk(input=dot_scores, 
                                 k=n_resources_to_return)

    return scores, indices

In [24]:
outputs_decoded = tokenizer.decode(outputs[0])

In [25]:
def formatter(query:str, context_item: list[dict]) -> str:
    context = "- " + "\n- ".join([item["sentence_chunk"] for item in context_item])
    base_prompt = """Based on the following context items, please answer the query.
Give yourself room to think by extracting relevant passages from the context before answering the query.
Don't return the thinking, only return the answer.
Make sure your answers are as explanatory as possible.
Use the following examples as reference for the ideal answer style.
REMEMBER NEVER TO USE THE FOLLOWING SENTENCES IN YOUR OUTPUT, THEY ATE JUST THE FORMATS
\nExample 1:
Query: What are the fat-soluble vitamins?
Answer: The fat-soluble vitamins include Vitamin A, Vitamin D, Vitamin E, and Vitamin K. These vitamins are absorbed along with fats in the diet and can be stored in the body's fatty tissue and liver for later use. Vitamin A is important for vision, immune function, and skin health. Vitamin D plays a critical role in calcium absorption and bone health. Vitamin E acts as an antioxidant, protecting cells from damage. Vitamin K is essential for blood clotting and bone metabolism.
\nExample 2:
Query: What are the causes of type 2 diabetes?
Answer: Type 2 diabetes is often associated with overnutrition, particularly the overconsumption of calories leading to obesity. Factors include a diet high in refined sugars and saturated fats, which can lead to insulin resistance, a condition where the body's cells do not respond effectively to insulin. Over time, the pancreas cannot produce enough insulin to manage blood sugar levels, resulting in type 2 diabetes. Additionally, excessive caloric intake without sufficient physical activity exacerbates the risk by promoting weight gain and fat accumulation, particularly around the abdomen, further contributing to insulin resistance.
\nExample 3:
Query: What is the importance of hydration for physical performance?
Answer: Hydration is crucial for physical performance because water plays key roles in maintaining blood volume, regulating body temperature, and ensuring the transport of nutrients and oxygen to cells. Adequate hydration is essential for optimal muscle function, endurance, and recovery. Dehydration can lead to decreased performance, fatigue, and increased risk of heat-related illnesses, such as heat stroke. Drinking sufficient water before, during, and after exercise helps ensure peak physical performance and recovery.
\nNow use the following context items to answer the user query:
{context}
\nRelevant passages: <extract relevant passages from the context here>
User query: {query}
Answer:"""
 
    base_prompt = base_prompt.format(context=context, query=query)

    # Create prompt template for instruction-tuned model
    dialogue_template = [
        {"role": "user",
        "content": base_prompt}
    ]

    # Apply the chat template
    prompt = tokenizer.apply_chat_template(conversation=dialogue_template,
                                          tokenize=False,
                                          add_generation_prompt=True)
    return prompt

query = ""

scores, indices = retrieve_relevant_resources(query=query, embeddings=embeddings)

context_item = [pac[i] for i in indices]

prompt = formatter(query=query, context_item=context_item)
prompt

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nBased on the following context items, please answer the query.\nGive yourself room to think by extracting relevant passages from the context before answering the query.\nDon\'t return the thinking, only return the answer.\nMake sure your answers are as explanatory as possible.\nUse the following examples as reference for the ideal answer style.\nREMEMBER NEVER TO USE THE FOLLOWING SENTENCES IN YOUR OUTPUT, THEY ATE JUST THE FORMATS\n\nExample 1:\nQuery: What are the fat-soluble vitamins?\nAnswer: The fat-soluble vitamins include Vitamin A, Vitamin D, Vitamin E, and Vitamin K. These vitamins are absorbed along with fats in the diet and can be stored in the body\'s fatty tissue and liver for later use. Vitamin A is important for vision, immune function, and skin health. Vitamin D plays a critical role in ca

In [26]:
inputid = tokenizer(prompt, return_tensors="pt").to("cuda")

outputs = llmmodel.generate(**inputid,
                            temperature=0.7,
                            do_sample=True,
                            max_new_tokens=256)

output_text = tokenizer.decode(outputs[0])
print(f"Query: {query}")
print(f"RAG answer:\n{output_text.replace(prompt, '')}")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Query: tell me how did krishna advice people when it comes to lust, how to tackle it
RAG answer:
<|begin_of_text|>Krishna advises that lust can be tackled by being educated in Krishna consciousness from the very beginning of life. This education enables one to understand the natural love of God and prevents it from deteriorating into lust. When one is educated in Krishna consciousness, lust can become a friend instead of an enemy, and can even be transformed into the love of Godhead, the highest perfectional stage of human life.<|eot_id|>


In [28]:
def rag(query, 
        temperature=0.7,
        max_new_tokens=512,
        format_answer_text=True, 
        return_answer_only=True):
    """
    Takes a query, finds relevant resources/context and generates an answer to the query based on the relevant resources.
    """
    
    # Get just the scores and indices of top related results
    scores, indices = retrieve_relevant_resources(query=query,
                                                  embeddings=embeddings)
    
    # Create a list of context items
    context_items = [pac[i] for i in indices]

    # Add score to context item
    for i, item in enumerate(context_items):
        item["score"] = scores[i].cpu() # return score back to CPU 
        
    # Format the prompt with context items
    prompt = formatter(query=query,
                              context_item=context_item)
    
    # Tokenize the prompt
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generate an output of tokens
    outputs = llmmodel.generate(**input_ids,
                                 temperature=temperature,
                                 do_sample=True,
                                 max_new_tokens=max_new_tokens)
    
    # Turn the output tokens into text
    output_text = tokenizer.decode(outputs[0])

    if format_answer_text:
        # Replace special tokens and unnecessary help message
        output_text = output_text.replace(prompt, "").replace("<bos>", "").replace("<eos>", "").replace("Sure, here is the answer to the user query:\n\n", "")

    # Only return the answer without the context items
    if return_answer_only:
        return output_text
    
    return output_text, context_items

In [34]:
print(rag(query="what did bhrahma say about krishna"))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>The relevant passages are:

"Bhagavad-gétä gives us that general and specific knowledge of the self. The living entities are parts and parcels of the Lord, and therefore they are simply meant to serve the Lord. This consciousness is called Krishna consciousness."

"Bhagavad-gétä" is mentioned here as the source of knowledge, and it is implied that Lord Krishna is the one who imparts this knowledge.<|eot_id|>


In [35]:
import torch
torch.cuda.empty_cache()